In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
module = hub.KerasLayer("https://tfhub.dev/google/experts/bit/r50x1/in21k/spermatophyte/1")


In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)


In [4]:
data_dir

'./datasets/flower_photos'

In [5]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [6]:
list(data_dir.glob('*/*.jpg'))[:5]


[PosixPath('datasets/flower_photos/dandelion/7179487220_56e4725195_m.jpg'),
 PosixPath('datasets/flower_photos/dandelion/8740218495_23858355d8_n.jpg'),
 PosixPath('datasets/flower_photos/dandelion/14019781123_ea0f8722d4_n.jpg'),
 PosixPath('datasets/flower_photos/dandelion/14829055_2a2e646a8f_m.jpg'),
 PosixPath('datasets/flower_photos/dandelion/14084345111_8a4cb05a31.jpg')]

In [7]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [8]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [9]:
flowers_images_dict['roses'][:5]


[PosixPath('datasets/flower_photos/roses/3751835302_d5a03f55e8_n.jpg'),
 PosixPath('datasets/flower_photos/roses/2273917656_6d6c038283.jpg'),
 PosixPath('datasets/flower_photos/roses/4918137796_21f0922b0c_n.jpg'),
 PosixPath('datasets/flower_photos/roses/3500121696_5b6a69effb_n.jpg'),
 PosixPath('datasets/flower_photos/roses/4684127262_6c3346188d.jpg')]

In [10]:
str(flowers_images_dict['roses'][0])


'datasets/flower_photos/roses/3751835302_d5a03f55e8_n.jpg'

In [11]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))


In [12]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [13]:
X = np.array(X)
y = np.array(y)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [15]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [16]:
# Now we will take a per traed model and rework on it 

feature_extractor_model = 'https://tfhub.dev/google/experts/bit/r50x1/in21k/spermatophyte/1'

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [17]:
num_of_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 23,510,597
Trainable params: 10,245
Non-trainable params: 23,500,352
_________________________________________________________________


In [18]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
86/86 [==============================] - 37s 199ms/step - loss: 0.5283 - acc: 0.8594
Epoch 2/2
86/86 [==============================] - 17s 202ms/step - loss: 0.1596 - acc: 0.9473


In [ ]:

model.evaluate(X_test_scaled,y_test)
